<a href="https://colab.research.google.com/github/zakrzewki/pracaMgr/blob/master/Mgr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First of all, take fresh data from repo

**Connect to Google Drive**

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
!git clone

Fast checker if connection and exporting files to Google Drive are working correctly

In [0]:
!python3 /content/pracaMgr/GoogleDriveChecker.py

**Downgrade keras** (`train_frcnn.py` throws error when it's 2.2.4 version)

In [0]:
!pip install keras==2.1.6

**Create a connection with Kaggle DB for downloading data**

In [0]:
!kaggle config set -n path -v{/content}

In [0]:
!mkdir /content/.kaggle

In [0]:
import json

token = {"username":"TOKEN","key":"KEY"}

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

Add folders necessary for dividing test and train data

In [0]:
!mkdir /content/pracaMgr/input/

In [0]:
!mkdir /content/pracaMgr/input/test_v2

In [0]:
!mkdir /content/pracaMgr/input/train_v2

Download data from Kaggle using following commands:

In [0]:
!kaggle competitions download -c airbus-ship-detection -p /content/pracaMgr/input

Unpack all data to selected folder

In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/pracaMgr/input/train_ship_segmentations_v2.csv.zip", 'r')
zip_ref.extractall("/content/pracaMgr/input/")
zip_ref.close()

In [0]:
zip_ref = zipfile.ZipFile("/content/pracaMgr/input/test_v2.zip", 'r')
zip_ref.extractall("/content/pracaMgr/input/test_v2")
zip_ref.close()

In [0]:
zip_ref = zipfile.ZipFile("/content/pracaMgr/input/train_v2.zip", 'r')
zip_ref.extractall("/content/pracaMgr/input/train_v2")
zip_ref.close()

Move unnecessary zip folders to somewhere else

In [0]:
!mkdir /kaggle_zips

In [0]:
import os

os.rename("/content/pracaMgr/input/train_ship_segmentations_v2.csv.zip", "/kaggle_zips/train_ship_segmentations_v2.csv.zip")

In [0]:
os.rename("/content/pracaMgr/input/train_v2.zip", "/kaggle_zips/train_v2.zip")

In [0]:
os.rename("/content/pracaMgr/input/test_v2.zip", "/kaggle_zips/test_v2.zip")

Using RLE algorithm take some example data for creating Bounding Boxes for training purposes

In [0]:
!python3 /content/pracaMgr/Decoder_onlyAligned.py --no_small 100

In [0]:
!python3 /content/pracaMgr/Background_decoder.py -n 10

In [0]:
!python3 /content/pracaMgr/Decoder_allShips.py --no_small 100

**Move it to the correct folder**



In [0]:
import os
if os.path.exists("/content/entry_data.csv"):
  os.rename("/content/entry_data.csv", "/content/pracaMgr/entry_data.csv")
  print("Moved")

**Train stuff**

In [0]:
!python3 /content/pracaMgr/train_frcnn.py simple -p /content/pracaMgr/entry_data.csv --num_epochs=40 --epoch_length 500 --num_rois 64

In [0]:
if os.path.exists("/content/entry_data.csv"):
  os.rename("/content/entry_data.csv", "/content/pracaMgr/entry_data.csv")
  print("Moved")

**Move weights file**

In [0]:
import shutil, os
if os.path.exists("/content/pracaMgr/input/weights.hdf5"): 
  shutil.copy("/content/pracaMgr/input/weights.hdf5", "/content/weights.hdf5")
  print("Copied weights from input")
if os.path.exists("/content/pracaMgr/input/config.pickle"): 
  shutil.copy("/content/pracaMgr/input/config.pickle", "/content/config.pickle")
  print("Copied config from input")
if os.path.exists("/content/pracaMgr/config.pickle"): 
  shutil.copy("/content/pracaMgr/config.pickle", "/content/config.pickle")
  print("Copied config")

Copied weights from input


Pull any changes new changes, if it's necessary

In [0]:
%cd /content/pracaMgr
!git pull

**Test it, how it's working**

In [0]:
!python3 /content/pracaMgr/test_frcnn.py --p /content/pracaMgr/input/test_v2/ --input_weight_path /content/weights.hdf5 --config_filename /content/config.pickle  --num_rois 64